# Tugas Besar 2 Pemrograman Lanjut Python

## Deskripsi


Anda bekerja pada Perusahaan Software house yang sedang mengerjakan aplikasi perpustakaan yang berbasis microservice. Technical leader anda meminta Anda membuat class dari python. Buatlah class Bernama buku yang mewakili setiap buku di perpustakaan. Setiap buku memiliki atribut berikut:

- Judul: berisi judul dari buku
- Penulis: berisi nama penulis
- Penerbit
- Tahun terbit
- Konten: Biasanya berisikan judul bab
- Iktisar: merupakan intisari dari buku, yang berisikan gagasan buku yang ditulis dengan menggunakan Bahasa sendiri.

Kelas buku harus mempunyai initializer yang dapat menerima judul buku, penulis dan konten buku sebagai argument. Nilai-nilai ini harus ditetapkan ke masing-masing atribut obyek.

Kelas tersebut juga harus memiliki method seperti di bawah ini:

- `read()`: method ini dapat mengambil nomor halaman, dan mencetak konten setiap halaman mulai dari dimulai sampai nomor halaman yang ditentukan tercapai
- `__str__()`: method ini mengembalikan string yang menjelaskan buku ini dalam hal nama dan penulisnya. (Contoh: Mathematics for Machine Learning by Marc Peter Deisenroth).


### Pertanyaan


a. Buatlah kelas yang memuat semua atribut di atas.
b. Buatlah tabel buku yang berisi ketiga atribut pada sistem basis data.
c. Buatlah method get yang mengambil data dari basis data yang telah anda buat.
d. Buatlah method post yang dapat menyimpan kelas buku.
e. Buatlah logger dan httpexception.


In [ ]:
# Import library yang dibutuhkan
!pip install fastapi uvicorn nest_asyncio pydantic mysql-connector-python

import mysql.connector
from fastapi import FastAPI, HTTPException
import logging
import uvicorn
import nest_asyncio

In [ ]:
# Siapkan logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:
# Konfigurasi database
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'port': '3306',
    'database': 'perpustakaan'
}


In [ ]:
from pydantic import BaseModel
from typing import List

# Class definition using Pydantic
class Buku(BaseModel):
    judul: str
    penulis: str
    konten: List[str]
    penerbit: str = ""
    tahun_terbit: str = ""
    iktisar: str = ""

    def read(self, halaman: int):
        if halaman > len(self.konten):
            halaman = len(self.konten)
        return self.konten[:halaman]

    def __str__(self):
        return f"{self.judul} by {self.penulis}"

In [ ]:
# Operasi database
def create_table():
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS buku (
        id INT AUTO_INCREMENT PRIMARY KEY,
        judul VARCHAR(255),
        penulis VARCHAR(255),
        konten TEXT,
        penerbit VARCHAR(255),
        tahun_terbit VARCHAR(4),
        iktisar TEXT
    )
    """)
    conn.commit()
    cursor.close()
    conn.close()


In [ ]:
# Fungsi penambahan buku
def add_buku(buku):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    cursor.execute("""
    INSERT INTO buku (judul, penulis, konten, penerbit, tahun_terbit, iktisar)
    VALUES (%s, %s, %s, %s, %s, %s)
    """, (buku.judul, buku.penulis, ",".join(buku.konten), buku.penerbit, buku.tahun_terbit, buku.iktisar))
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
# Fungsi pengambilan buku
def get_buku(judul):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT * FROM buku WHERE judul = %s", (judul,))
    result = cursor.fetchone()
    cursor.close()
    conn.close()
    if result:
        result['konten'] = result['konten'].split(',')
    return result

In [ ]:
# Aplikasi FastAPI
app = FastAPI()

@app.post("/buku/")
def create_buku(buku: Buku):
    try:
        add_buku(buku)
        logger.info(f"Buku '{buku.judul}' berhasil ditambahkan.")
        return {"message": "Buku berhasil ditambahkan"}
    except Exception as e:
        logger.error(f"Error adding buku: {str(e)}")
        raise HTTPException(status_code=500, detail="Terjadi kesalahan saat menambahkan buku")

@app.get("/buku/{judul}")
def read_buku(judul: str):
    try:
        buku = get_buku(judul)
        if buku:
            return buku
        else:
            raise HTTPException(status_code=404, detail="Buku tidak ditemukan")
    except Exception as e:
        logger.error(f"Error reading buku: {str(e)}")
        raise HTTPException(status_code=500, detail="Terjadi kesalahan saat mengambil data buku")


In [ ]:
# Buat table buku
create_table()

In [ ]:
# Jalankan aplikasi FastAPI
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)